In [1]:
import torch.nn.functional as F
import torch
import numpy as np
from pathlib import Path
import plotly.graph_objs as go


from initialization import NeuralSurfaceReconstruction

In [2]:
nsr = NeuralSurfaceReconstruction(
    data=Path("../data/calculator"),
)

In [15]:
_, st = nsr.query_pixel_sample_points(700, N_query_points=100, t_size=0.15,
                                        _pixel_grid_width_step=100.0,
                                        _pixel_grid_height_step=100.0,
                                        _scale=1.0)
stx, sty, stz = np.hsplit(st.reshape(-1, 3), 3)

In [29]:
st_i = st[:, -1]

In [30]:
x_i = torch.tensor(st_i).float()
sdf_i = nsr.model.S(x_i)
# nsr.model.fg_V.sdf
# x_i.float()
sdf_i

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 

In [31]:
x_i

tensor([[ -5.3672,   6.3072, -12.4630],
        [ -5.2560,   6.5579, -12.1208],
        [ -5.1451,   6.8073, -11.7807],
        ...,
        [ -8.5112,  12.5739,   3.6718],
        [ -8.4001,  12.8229,   4.0106],
        [ -8.2888,  13.0730,   4.3515]])

In [7]:
ft, bt = nsr.model.sector(x_i)

# ft = ((nsr.fg_V.min_x < st_i[:, 0]) & (st_i[:, 0] < nsr.fg_V.max_x) &
#  (nsr.fg_V.min_y < st_i[:, 1]) & (st_i[:, 1] < nsr.fg_V.max_y) &
#  (nsr.fg_V.min_z < st_i[:, 2]) & (st_i[:, 2] < nsr.fg_V.max_z))

fst = st_i[ft]
fstx, fsty, fstz = np.hsplit(fst.reshape(-1, 3), 3)

# bt = ((nsr.bg_V.min_x < st_i[:, 0]) & (st_i[:, 0] < nsr.bg_V.max_x) &
#  (nsr.bg_V.min_y < st_i[:, 1]) & (st_i[:, 1] < nsr.bg_V.max_y) &
#  (nsr.bg_V.min_z < st_i[:, 2]) & (st_i[:, 2] < nsr.bg_V.max_z))

bst = st_i[bt & ~ft]  #*bt[:, np.newaxis]*(1-ft[:, np.newaxis])
bstx, bsty, bstz = np.hsplit(bst.reshape(-1, 3), 3)

In [8]:
rfst = np.floor((fst-nsr.fg_V.voxel_pos[:, 0])/nsr.fg_V.step_size)*nsr.fg_V.step_size+nsr.fg_V.voxel_pos[:, 0]
rbst = np.floor((bst-nsr.bg_V.voxel_pos[:, 0])/nsr.bg_V.step_size)*nsr.bg_V.step_size+nsr.bg_V.voxel_pos[:, 0]

In [9]:
nfst = (fst-nsr.fg_V.center)/nsr.fg_V.scale
fg_npts = torch.flip(
    torch.Tensor(nfst[np.newaxis, np.newaxis, np.newaxis]),
    dims=[-1]
)
fg_sdf = F.grid_sample(nsr.fg_V.sdf, fg_npts,
                           mode="bilinear", padding_mode="border", align_corners=True).flatten()

nbst = ((bst-nsr.bg_V.center)/nsr.bg_V.scale)
bg_npts = torch.flip(
    torch.Tensor(nbst[np.newaxis, np.newaxis, np.newaxis]),
    dims=[-1]
)
bg_sdf = F.grid_sample(nsr.bg_V.sdf, bg_npts,
                           mode="bilinear", padding_mode="border", align_corners=True).flatten()

In [10]:
len(st_i), len(fg_sdf)+len(bg_sdf), len(fg_sdf), len(bg_sdf)

(858, 858, 0, 858)

In [32]:
go.Figure(data=[
    go.Scatter3d(
        x=nsr.fg_V.grid_pos[0].flatten(),
        y=nsr.fg_V.grid_pos[1].flatten(),
        z=nsr.fg_V.grid_pos[2].flatten(),
        mode='markers',
        marker={
            'size': 10,
            "color": nsr.fg_V.sdf.flatten(),
            'opacity': 0.2,
            "cmax": 1.0,
            "cmin": -1.0,
            "colorbar": {
                "thickness": 10,
            }
        }
    ),
    go.Scatter3d(
        x=nsr.bg_V.grid_pos[0].flatten(),
        y=nsr.bg_V.grid_pos[1].flatten(),
        z=nsr.bg_V.grid_pos[2].flatten(),
        mode='markers',
        marker={
            'size': 10,
            "color": nsr.bg_V.sdf.flatten(),
            'opacity': 0.3,
            "cmax": 1.0,
            "cmin": -1.0,
            "colorbar": {
                "thickness": 10,
            }
        }
    ),
    go.Scatter3d(
        x=st_i[:, 0],
        y=st_i[:, 1],
        z=st_i[:, 2],
        mode='markers',
        marker={
            'size': 3,
            "color": sdf_i,
            'opacity': 1.0,
            "cmax": 1.0,
            "cmin": -1.0,
        }
    ),
    # go.Scatter3d(
    #     x=stx.flatten(),
    #     y=sty.flatten(),
    #     z=stz.flatten(),
    #     mode='markers',
    #     marker={
    #         'size': 1,
    #         'opacity': 0.2,
    #     }
    # ),
    # go.Scatter3d(
    #     x=fst[:, 0],
    #     y=fst[:, 1],
    #     z=fst[:, 2],
    #     mode='markers',
    #     marker={
    #         'size': 3,
    #         "color": fg_sdf,
    #         'opacity': 1.0,
    #         "cmax": 1.0,
    #         "cmin": -1.0,
    #     }
    # ),
    # go.Scatter3d(
    #     x=bst[:, 0],
    #     y=bst[:, 1],
    #     z=bst[:, 2],
    #     mode='markers',
    #     marker={
    #         'size': 3,
    #         "color": bg_sdf,
    #         'opacity': 1.0,
    #         "cmax": 1.0,
    #         "cmin": -1.0,
    #     }
    # ),
    # go.Scatter3d(
    #     x=bstx.flatten(),
    #     y=bsty.flatten(),
    #     z=bstz.flatten(),
    #     mode='markers',
    #     marker={
    #         'size': 3,
    #         'opacity': 0.9,
    #     }
    # ),
], layout=go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0},
    scene=dict(
        aspectmode="cube",
        xaxis=dict(range=[-30, 30]),  # Set x-axis range
        yaxis=dict(range=[-30, 30]),  # Set y-axis range
        zaxis=dict(range=[-30, 30])   # Set z-axis range
    ),
    scene_camera= dict(
        eye=dict(x=0.0, y=0.0, z=0.1)
    )
))

In [116]:
a = np.arange(0, 10)
b = a[[2,4,6]]+1
a[[1,2]] = b
a

ValueError: shape mismatch: value array of shape (3,) could not be broadcast to indexing result of shape (2,)

In [121]:
nsr.fg_V.sdf.shape

torch.Size([1, 1, 16, 16, 16])

In [179]:
nsr.fg_V.grid_pos[0, :, 0, 0]

array([-3.25, -3.05, -2.85, -2.65, -2.45, -2.25, -2.05, -1.85, -1.65,
       -1.45, -1.25, -1.05, -0.85, -0.65, -0.45, -0.25])

In [180]:
np.array([
    nsr.fg_V.grid_pos[0, :, 0, 0], nsr.fg_V.grid_pos[1, 0, :, 0], nsr.fg_V.grid_pos[2, 0, 0, :]
])

array([[-3.25, -3.05, -2.85, -2.65, -2.45, -2.25, -2.05, -1.85, -1.65,
        -1.45, -1.25, -1.05, -0.85, -0.65, -0.45, -0.25],
       [ 0.  ,  0.2 ,  0.4 ,  0.6 ,  0.8 ,  1.  ,  1.2 ,  1.4 ,  1.6 ,
         1.8 ,  2.  ,  2.2 ,  2.4 ,  2.6 ,  2.8 ,  3.  ],
       [-1.  , -0.8 , -0.6 , -0.4 , -0.2 ,  0.  ,  0.2 ,  0.4 ,  0.6 ,
         0.8 ,  1.  ,  1.2 ,  1.4 ,  1.6 ,  1.8 ,  2.  ]])

In [181]:
FG_CENTER = np.array([
    (nsr.fg_V.grid_pos[0, :, 0, 0].max()+nsr.fg_V.grid_pos[0, :, 0, 0].min())/2,
    (nsr.fg_V.grid_pos[1, 0, :, 0].max()+nsr.fg_V.grid_pos[1, 0, :, 0].min())/2,
    (nsr.fg_V.grid_pos[2, 0, 0, :].max()+nsr.fg_V.grid_pos[2, 0, 0, :].min())/2,
])
FG_FACTOR = np.array([
    (nsr.fg_V.grid_pos[0, :, 0, 0].max()-nsr.fg_V.grid_pos[0, :, 0, 0].min())/2,
    (nsr.fg_V.grid_pos[1, 0, :, 0].max()-nsr.fg_V.grid_pos[1, 0, :, 0].min())/2,
    (nsr.fg_V.grid_pos[2, 0, 0, :].max()-nsr.fg_V.grid_pos[2, 0, 0, :].min())/2,
])

In [182]:
FG_CENTER, FG_FACTOR

(array([-1.75,  1.5 ,  0.5 ]), array([1.5, 1.5, 1.5]))

In [183]:
nsr.fg_V.sdf

tensor([[[[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 0.9900,  ..., 1.0000, 1.0000, 1.0000],
           ...,
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

          [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           ...,
           [1.0000, 1.0000, 0.9900,  ..., 0.9900, 1.0000, 1.0000],
           [0.9900, 1.0000, 0.9900,  ..., 1.0000, 1.0000, 1.0000],
           [0.9900, 1.0000, 0.9900,  ..., 1.0000, 1.0000, 1.0000]],

          [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000

In [188]:
nfst = ((fst-FG_CENTER)/FG_FACTOR)
norm_points = torch.Tensor(nfst[np.newaxis, np.newaxis, np.newaxis])
# norm_points

In [189]:
density = F.grid_sample(nsr.fg_V.sdf, norm_points, mode="bilinear", align_corners=True)

In [190]:
density[0][0][0][0]

tensor([ 1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  0.9971,  0.9894,
         0.9883,  0.9937,  1.0000,  1.0000,  1.0000,  0.9985,  0.9969,  0.9996,
         1.0000,  0.9984,  0.9961,  0.9994,  1.0000,  1.0000,  1.0000,  1.0000,
         1.0000,  0.9999,  0.9935,  0.9774,  0.9859,  0.9939,  0.9995,  0.9994,
         0.9998,  0.9963,  0.9922,  0.9983,  0.9997,  0.9996,  0.9991,  0.9998,
         1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  0.9983,  0.9955,  0.9921,
         0.9962,  0.9963,  0.9948,  0.9947,  0.9982,  0.9993,  0.9978,  0.9948,
         0.9977,  0.9971,  0.9923,  0.9986,  1.0000,  1.0000,  1.0000,  1.0000,
         1.0000,  0.9990,  0.9955,  0.9884,  0.9955,  0.9950,  0.9927,  0.9967,
         0.9990,  1.0000,  0.9996,  0.9974,  0.9988,  0.9986,  0.9961,  0.9993,
         1.0000,  1.0000,  1.0000,  0.9999,  0.9981,  0.9962,  0.9907,  0.9792,
         0.9900,  0.9921,  0.9920,  0.9993,  1.0000,  0.9992,  0.9982,  0.9997,
         0.9995,  0.9992,  0.9992,  0.99

In [133]:
nfst[:, 0]

array([-0.94425242, -0.94220892, -0.94029306, -0.93850434, -0.93684229,
       -0.93530641, -0.93389622, -0.93261121, -0.93145092, -0.93041484,
       -0.92950249, -0.92871337, -0.92804701, -0.92750291, -0.92708058,
       -0.92677954, -0.92659929, -0.92653935, -0.92659923, -0.92677843,
       -0.92707647, -0.85103638, -0.84929155, -0.84764887, -0.84610787,
       -0.84466804, -0.84332892, -0.84208999, -0.84095079, -0.83991081,
       -0.83896957, -0.83812658, -0.83738135, -0.8367334 , -0.83618223,
       -0.83572735, -0.83536828, -0.83510453, -0.8349356 , -0.83486102,
       -0.83488029, -0.83499291, -0.7581124 , -0.75666607, -0.75529641,
       -0.75400295, -0.7527852 , -0.75164266, -0.75057485, -0.74958127,
       -0.74866145, -0.74781489, -0.7470411 , -0.7463396 , -0.74570988,
       -0.74515148, -0.74466389, -0.74424663, -0.74389921, -0.74362114,
       -0.74341193, -0.7432711 , -0.74319815, -0.66540404, -0.66425604,
       -0.66315925, -0.66211317, -0.66111732, -0.6601712 , -0.65

In [159]:
nsr.fg_V.sdf

tensor([[[[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 0.9900,  ..., 1.0000, 1.0000, 1.0000],
           ...,
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

          [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           ...,
           [1.0000, 1.0000, 0.9900,  ..., 0.9900, 1.0000, 1.0000],
           [0.9900, 1.0000, 0.9900,  ..., 1.0000, 1.0000, 1.0000],
           [0.9900, 1.0000, 0.9900,  ..., 1.0000, 1.0000, 1.0000]],

          [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000

In [89]:
fst.tolist()

[[-3.1663786267686573, 0.11171259732649119, 0.9454343539553438],
 [-3.1633133758755663, 0.25589379188929584, 0.9435643221953125],
 [-3.160439582740156, 0.3987926520041585, 0.941713302006578],
 [-3.1577565141399218, 0.5405238306441356, 0.9398798310385903],
 [-3.155263436852362, 0.6812019807822841, 0.9380624469408101],
 [-3.1529596176549735, 0.8209417553916609, 0.9362596873626892],
 [-3.1508443233252525, 0.9598578074453226, 0.9344700899536793],
 [-3.148916820640697, 1.098064789916326, 0.932692192363243],
 [-3.147176376378803, 1.2356773557777276, 0.9309245322408284],
 [-3.1456222573170676, 1.3728101580025844, 0.9291656472358945],
 [-3.1442537302329883, 1.509577849563953, 0.9274140749978965],
 [-3.1430700619040612, 1.64609508343489, 0.9256683531762864],
 [-3.1420705191077842, 1.7824765125884525, 0.9239270194205211],
 [-3.141254368621654, 1.9188367899976968, 0.9221886113800544],
 [-3.140620877223167, 2.0552905686356797, 0.9204516667043414],
 [-3.140169311689821, 2.191952501475458, 0.9187147

In [112]:
nfst

[[-0.9442524178457715, -0.9255249351156726, 0.2969562359702292],
 [-0.9422089172503775, -0.8294041387404695, 0.29570954813020833],
 [-0.940293055160104, -0.7341382319972277, 0.294475534671052],
 [-0.9385043427599479, -0.6396507795705763, 0.2932532206923935],
 [-0.936842291234908, -0.5458653461451439, 0.2920416312938734],
 [-0.9353064117699823, -0.4527054964055594, 0.2908397915751261],
 [-0.9338962155501683, -0.36009479503645164, 0.2896467266357862],
 [-0.9326112137604646, -0.2679568067224493, 0.2884614615754953],
 [-0.9314509175858688, -0.17621509614818157, 0.28728302149388557],
 [-0.9304148382113784, -0.08479322799827704, 0.2861104314905963],
 [-0.9295024868219922, 0.006385233042635274, 0.28494271666526433],
 [-0.9287133746027075, 0.09739672228992664, 0.28377890211752427],
 [-0.9280470127385229, 0.18831767505896835, 0.2826180129470141],
 [-0.927502912414436, 0.2792245266651312, 0.2814590742533696],
 [-0.9270805848154448, 0.37019371242378646, 0.28030111113622763],
 [-0.9267795411265473